In [7]:

from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from opensearchpy.exceptions import TransportError, ConnectionError as OSCxnError

from requests_aws4auth import AWS4Auth
from openai import OpenAI, OpenAIError
from config import *
import time
import logging

from src.services.querying import llm_client, os_client, reconstruct_thread, answer_query, knn_search, Memory


In [8]:
# query = "How much do we ususally charge for traspots for 'compassionate funerals'?"

# hits, ids, q_vec = knn_search(query)

# for i , hit in enumerate(hits, start=1):
#     print(f"----Email thread {i}----")
#     print()
#     print(f"ID: {hit["_id"]}")
#     print(f"Score: {hit["_score"]}")
#     print(f"Subject: {hit["_source"]["subject"]}")
#     print(f"Summary: {hit["_source"]["summary_text"]}")
#     print()

In [9]:
retrieved_ids = []
memory = Memory(
    llm_client=llm_client,
    os_client=os_client,
    short_term_tokens=1000,
    mid_term_turns=3,
    memory_model=MEMORY_MODEL,
    embeddings_model=EMBEDDINGS_MODEL
)

In [10]:
bank=[
 "What how does Redcoat work with Lufthansa?",
 "Can you give specific examples?",
 "What are Lufthansa's rates to Africa?",
 "And what about other destinations, like US?",
 "Who is Simon and his role?",
 "How does he collaborate with Sarah Hawkins?",
 "Who else works there?",
 "If we were to compare US and African imports to the UK. How do they compare?",
 "Draft an SOP to import goods to the UK."
 ]

#query = bank[-1]
query = "HOw much do we charge customer for HUM from UK to gambia"
retrieved_ids = []  

prompt, response, mem_ctx, retrieved_ids, q_vec = answer_query(query, retrieved_ids=retrieved_ids, memory=memory)



[answer_query] Turn 1 | mem_summary='' | last_snip=''
[answer_query] Rewritten query: 'What is the cost of shipping HUM from the UK to Gambia for customers?'
[answer_query] Calling knn_search with retrieved_ids=[]
[answer_query] knn_search returned 5 hits | new retrieved_ids=['t_0000000000000f6e4e05f650738a@google.com', 't_ca+y-08ha7r1tdk-xgqmg0f5m8cwsyp_fcagn3pnf3-kwt92chq@mail.gmail.com', 't_loyp123mb3360bef838813d05fc9132c9beb49@loyp123mb3360.gbrp123.prod.outlook.com', 't_ca+y-08gxfhmzwffogfldoqdod6pzvwmifryhg8w1gwfu9k2erq@mail.gmail.com', 't_ca+y-08frxo+spqkjmzec8tbu-_wc04nv7qax7okmjce+ld_qaw@mail.gmail.com']
[answer_query] Formatted thread_blocks length: 35070
[answer_query] Retrieved long-term facts count: 0
[answer_query] Built system_msgs with 3 messages
[answer_query] Prompt tokens=11077 | selected_model=gpt-3.5-turbo-16k
[answer_query] Sending request to OpenAI model gpt-3.5-turbo-16k...
[answer_query] Received response (length=1251)
[answer_query] Updated short-term and mid-

In [11]:
import tiktoken

enc = tiktoken.encoding_for_model(QUERY_MODEL)

print(len(enc.encode(prompt)))

11077


In [12]:
#print(response)
print(response)

Based on the email threads provided, the cost for shipping HUMs from Banjul to the UK on ZT was GBP1750.00. This cost was confirmed by Simon in response to Lloyd's email. The breakdown of the cost included GBP131250.00 if selling locally in Gambia. Additionally, for future UK payments, an invoice of GMD32812.50 for profit split was required.

In a later email exchange, there was a discussion about the cost of sending HUMs on SN, with Mansour mentioning a cost of D178,692.00 based on 150 kgs. Lloyd suggested a selling cost of GBP1800.00 for the shipment. The conversation also involved coordinating billing details for shipments to Banjul and GAMEXP, with a profit-sharing arrangement. Lloyd mentioned charging GAMEXP GBP1800, leaving a profit of GBP900 to be split or kept based on a 50/50 split or a different arrangement.

Simon inquired about billing Banjul for the shipment, suggesting putting the Airfreight sale as one line at GBP1200. Lloyd confirmed this approach, indicating it was fin

Conditional Retrieval

Cooldown TLL

Freshness Boosting

MRR (Penalizing documents which are too similar to what we already have)

Hierarchical Memory